In [2]:
import sklearn.metrics
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
from collections import OrderedDict
from sklearn.model_selection import train_test_split
import  matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import numpy as np
import pandas as pd



In [3]:
def MAPE_regression(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 

In [62]:
#NuralNet (Regression)
class RegressionNet(nn.Module):
    def __init__(self, X_train, hiddenlayer, node, Ouputnode):
        super().__init__() #nn.Module의 __init__속성 및 메소드를 불러옴
        self.X_train = X_train
        self.hiddenlayer = hiddenlayer
        self.node = node
        self.Ouputnode = Ouputnode
        ordered_dict = OrderedDict()
        ordered_dict['Linear1'] = nn.Linear(X_train.size(1) , node)
        ordered_dict['relu1'] = nn.ReLU()
        for i in range(2 , hiddenlayer+2 ):
            ordered_dict['Linear{}'.format(i)] = nn.Linear(node, node)
            ordered_dict['relu{}'.format(i)] = nn.ReLU()
        ordered_dict['Linear{}'.format(hiddenlayer+2)] = nn.Linear(node, Ouputnode)
        self.layer1  = nn.Sequential(ordered_dict)
        
        
    def predict(self, X_test):
        pred_x = self.layer1(X_test)
        
        return {"Predarray" :pred_x}

    def fit(self,epochs,loader, criterion, optimizer):
        losses = []
        for epoch in range(epochs):
            for step, (x, label) in enumerate(loader):
                optimizer.zero_grad()  # optimizer의 매개변수를 0으로 만듬
                y_pred = self.layer1(x)

                loss = criterion(y_pred, label)
                losses.append(loss)
                loss.backward()
                optimizer.step()
            if epoch % 10 == 0:
                print("{}epoch // loss ={}".format(epoch, loss))
                
   
        

In [63]:
#NuralNet (Classification)
class ClassificationNet(nn.Module):
    def __init__(self, X_train, hiddenlayer, node, Ouputnode):
        super().__init__() #nn.Module의 __init__속성 및 메소드를 불러옴
        self.X_train = X_train
        self.hiddenlayer = hiddenlayer
        self.node = node
        self.Ouputnode = Ouputnode
        ordered_dict = OrderedDict()
        ordered_dict['Linear1'] = nn.Linear(X_train.size(1) , node)
        ordered_dict['relu1'] = nn.ReLU()
        for i in range(2 , hiddenlayer+2 ):
            ordered_dict['Linear{}'.format(i)] = nn.Linear(node, node)
            ordered_dict['relu{}'.format(i)] = nn.ReLU()
        ordered_dict['Linear{}'.format(hiddenlayer+2)] = nn.Linear(node, Ouputnode)
        self.layer1  = nn.Sequential(ordered_dict)
        
        
    def predict(self, X_test):
        pred_x = self.layer1(X_test)
        
        return {"Predarray" :pred_x}

    
    def fit(self,epochs,loader, criterion, optimizer):
        losses = []
        for epoch in range(epochs):
            for step, (x, label) in enumerate(loader):
                optimizer.zero_grad()  # optimizer의 매개변수를 0으로 만듬
                y_pred = self.layer1(x)

                loss = criterion(y_pred, label)
                losses.append(loss)
                loss.backward()
                optimizer.step()
            if epoch % 10 == 0:
                print("{}epoch // loss ={}".format(epoch, loss))

In [64]:

def CNet(X_train, y_train, X_test,y_test, hiddenlayer, node, Ouputnode, epochs):
    '''
    
        @Input 
            X_train : type : array
            y_train : type : array
            X_test : type : array
            hiddenlayer : type : int
            node : type : int
            Ouputnode : type : int
            epochs : type : int
            
        @Output
            pred_y_test : type : array
            
    '''
    
    
    # 데이터를 파이토치 텐서로 변경
    X = torch.Tensor(X_train.values)
    Y = torch.Tensor(y_train.values)
    X_test = torch.Tensor(X_test.values)
    
    # 학습을 위한 데이터 전처리 
    dataset = TensorDataset(X, Y)
    loader = DataLoader(dataset, batch_size=1, shuffle=True)
    
    #신경망 생성하기
    model = ClassificationNet(X, hiddenlayer, node, Ouputnode)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    model.fit(epochs,loader, criterion, optimizer)
    
    
    #학습되 모델 저장
    torch.save(model, "CNet.pt")
    
    #테스트 데이터로 예측 결과 반환
    pred_y_test = model.predict(X_test)
    pred_y_test_tensor=pred_y_test['Predarray']
    print(pred_y_test_tensor)
    #파이토치 텐서를 넘파이 어레이로 전환
    pred_y_test_tensor = pred_y_test_tensor.detach().numpy()
    
      # calculate AUC of model
    fpr, tpr, thresholds = roc_curve(y_test, pred_y_test_tensor)
    auc = roc_auc_score(y_test, pred_y_test_tensor)
      
    return  {"auc_score": auc, "fpr":fpr, "tpr":tpr}
    

In [65]:
def RNet(X_train, y_train, X_test, y_test , hiddenlayer, node, Ouputnode, epochs):
    
    '''
    
        @Input 
            X_train : type : array
            y_train : type : array
            X_test : type : array
            hiddenlayer : type : int
            node : type : int
            Ouputnode : type : int
            epochs : type : int
            
        @Output
            pred_y_test : type : array
            
    '''
    
    # 데이터를 파이토치 텐서로 변경
    X = torch.Tensor(X_train.values)
    Y = torch.Tensor(y_train.values)
    X_test = torch.Tensor(X_test.values)
    
    # 학습을 위한 데이터 전처리 
    dataset = TensorDataset(X, Y)
    loader = DataLoader(dataset, batch_size=1, shuffle=True)

    # 신경망 생성하기
    model = RegressionNet(X, hiddenlayer, node, Ouputnode)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.fit(epochs,loader, criterion, optimizer)
            
    #학습되 모델 저장
    torch.save(model, "RNet.pt")
    
    #테스트 데이터로 예측 결과 반환
    pred_y_test = model.predict(X_test)
    pred_y_test_tensor = pred_y_test['Predarray'] 
    #파이토치 텐서를 넘파이 어레이로 전환
    pred_y_test = pred_y_test.detach().numpy()
    
    MSE = mean_squared_error(y_test, pred_y_test)  # MSE 값은 출력하지 않아도 됩니다. RMSE를 구하기 위한 절차입니다.
    
    # 출력 대상입니다. 성능지표로써 3가지값이 출력되게 됩니다.
    RMSE = np.sqrt(MSE)
    MAPE = MAPE_regression(y_test, pred_y_test)
    R_squared = r2_score(y_test, pred_y_test)
    return {'pred_y' : pred_y_test, 'RMSE' : RMSE, 'MAPE' : MAPE, "R_squared" : R_squared}

In [66]:
# ANN Model load

def ANN_model_loader(pickleFile, tsdat) :
    """
    저장한 모델을 로드한 후, 로드한 모델과 데이터를 활용해 분석 결과 리턴

    Parameters
    ----------
    model : ?
        로드한 모델
    tsdat : array
        예측 데이터

    Returns
    -------
    모델 리턴과 동일

    """
    model = torch.load(pickleFile)  
    X = torch.Tensor(tsdat.values)
    pred = model.predict(X)
    pred = pred.detach().numpy()
    
    
    return {'pred' : pred }



In [67]:
df1 = pd.read_csv("ab_te.csv")
df2 = pd.read_csv("RMS_bearing.csv")

In [68]:
x = df1.iloc[:,[0, 1, 2, 3]]
y = df1.iloc[:,[4]]
x1 = df2.iloc[:,[0,1,2]]
y1 = df2.iloc[:,[3]]


X_train,X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=33)

X1_train,X1_test, y1_train, y1_test = train_test_split(x1,y1, test_size=0.2, random_state=33)

In [58]:
pred_y2_test= CNet(X_train, y_train, X_test,y_test, 2, 5, 1, 10)



#CNet(X_train, y_train, X_test,y_test, hiddenlayer, node, Ouputnode, epochs):

0epoch // loss =-0.0
tensor([[0.3893],
        [0.3502],
        [0.3089],
        [0.3216],
        [0.3214],
        [0.3107],
        [0.3192],
        [0.3122],
        [0.3167],
        [0.3288],
        [0.3338],
        [0.3363],
        [0.3205],
        [0.3327],
        [0.3071],
        [0.3160],
        [0.3085],
        [0.3128],
        [0.3124],
        [0.3525],
        [0.3266],
        [0.3111],
        [0.3172],
        [0.3483],
        [0.3197],
        [0.3068],
        [0.3165],
        [0.3283],
        [0.3152],
        [0.3646],
        [0.3234],
        [0.3163],
        [0.3105],
        [0.3135],
        [0.3238],
        [0.3300],
        [0.3082],
        [0.3426],
        [0.3181],
        [0.3651],
        [0.3337],
        [0.3112],
        [0.3244],
        [0.3173],
        [0.3086],
        [0.3053],
        [0.3369],
        [0.3245],
        [0.3206],
        [0.3189],
        [0.3380],
        [0.3296],
        [0.3332],
        [0.3193],
       

In [59]:
print(pred_y2_test)

{'auc_score': 0.5558488469019173, 'fpr': array([0.        , 0.        , 0.00819672, 0.00819672, 0.01639344,
       0.01639344, 0.04098361, 0.04098361, 0.07377049, 0.07377049,
       0.09016393, 0.09016393, 0.10655738, 0.10655738, 0.1147541 ,
       0.1147541 , 0.12295082, 0.12295082, 0.13114754, 0.13114754,
       0.13934426, 0.13934426, 0.14754098, 0.14754098, 0.16393443,
       0.16393443, 0.18852459, 0.18852459, 0.19672131, 0.19672131,
       0.2295082 , 0.2295082 , 0.23770492, 0.23770492, 0.26229508,
       0.26229508, 0.28688525, 0.28688525, 0.29508197, 0.29508197,
       0.30327869, 0.30327869, 0.31147541, 0.31147541, 0.31967213,
       0.31967213, 0.33606557, 0.33606557, 0.3442623 , 0.3442623 ,
       0.36065574, 0.36065574, 0.37704918, 0.37704918, 0.39344262,
       0.39344262, 0.40163934, 0.40163934, 0.40983607, 0.40983607,
       0.42622951, 0.42622951, 0.44262295, 0.44262295, 0.45081967,
       0.45081967, 0.46721311, 0.46721311, 0.47540984, 0.47540984,
       0.48360656, 0.

In [69]:
pred_y1_test = RNet(X1_train, y1_train, X1_test, y1_test, 2, 5, 1, 10)


0epoch // loss =1.4847279089735821e-05


AttributeError: 'dict' object has no attribute 'detach'

In [ ]:
print(pred_y1_test)

In [ ]:
RNET = torch.load("RNet.pt")
RNET.eval()

In [52]:
CNET = torch.load("CNet.pt")
CNET.eval()

ClassificationNet(
  (layer1): Sequential(
    (Linear1): Linear(in_features=4, out_features=5, bias=True)
    (relu1): ReLU()
    (Linear2): Linear(in_features=5, out_features=5, bias=True)
    (relu2): ReLU()
    (Linear3): Linear(in_features=5, out_features=5, bias=True)
    (relu3): ReLU()
    (Linear4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [53]:
pred = ANN_model_loader("RNet.pt", X1_test) 
pred2 = ANN_model_loader("CNet.pt", X_test) 

AttributeError: 'dict' object has no attribute 'detach'

In [16]:
print(pred)

{'pred': array([[0.0490789 ],
       [0.04988939],
       [0.05048024],
       [0.06414533],
       [0.06259635],
       [0.05126268],
       [0.05011994],
       [0.04925032],
       [0.06523835],
       [0.05114648],
       [0.04937062],
       [0.04970318],
       [0.04922732],
       [0.05044145],
       [0.04971346],
       [0.05043907],
       [0.04991469],
       [0.05559953],
       [0.05023248],
       [0.05501351],
       [0.05955158],
       [0.04986075],
       [0.05009352],
       [0.04950306],
       [0.05107646],
       [0.05277202],
       [0.05000123],
       [0.04957911],
       [0.04946515],
       [0.05044197],
       [0.05413874],
       [0.05234855],
       [0.05048864],
       [0.06600244],
       [0.01080777],
       [0.05621214],
       [0.04923608],
       [0.05049842],
       [0.05630585],
       [0.05028473],
       [0.04948408],
       [0.06013571],
       [0.04977857],
       [0.04980896],
       [0.04930496],
       [0.04999562],
       [0.05550478],
    

In [17]:
print(pred2)

{'pred': array([[0.06082285],
       [0.03531693],
       [0.06015452],
       [0.06647818],
       [0.09978382],
       [0.06402427],
       [0.06857114],
       [0.04694599],
       [0.02885805],
       [0.07244048],
       [0.04570845],
       [0.06111665],
       [0.06232505],
       [0.05776316],
       [0.04532118],
       [0.05221681],
       [0.0648113 ],
       [0.07605117],
       [0.05043478],
       [0.05442171],
       [0.06589334],
       [0.06144997],
       [0.07230978],
       [0.08978073],
       [0.08569213],
       [0.08341805],
       [0.03744788],
       [0.07439743],
       [0.02822126],
       [0.06632197],
       [0.06137627],
       [0.06748128],
       [0.06025125],
       [0.05608419],
       [0.03358889],
       [0.0572008 ],
       [0.08724836],
       [0.04561791],
       [0.04727604],
       [0.04373495],
       [0.02962787],
       [0.06444164],
       [0.06352842],
       [0.06731652],
       [0.07679373],
       [0.05718098],
       [0.05913889],
    

In [18]:
X_test = torch.Tensor(X_test.values)
pred_y_test = CNET.predict(X_test)
print(pred_y_test)

tensor([[0.0608],
        [0.0353],
        [0.0602],
        [0.0665],
        [0.0998],
        [0.0640],
        [0.0686],
        [0.0469],
        [0.0289],
        [0.0724],
        [0.0457],
        [0.0611],
        [0.0623],
        [0.0578],
        [0.0453],
        [0.0522],
        [0.0648],
        [0.0761],
        [0.0504],
        [0.0544],
        [0.0659],
        [0.0614],
        [0.0723],
        [0.0898],
        [0.0857],
        [0.0834],
        [0.0374],
        [0.0744],
        [0.0282],
        [0.0663],
        [0.0614],
        [0.0675],
        [0.0603],
        [0.0561],
        [0.0336],
        [0.0572],
        [0.0872],
        [0.0456],
        [0.0473],
        [0.0437],
        [0.0296],
        [0.0644],
        [0.0635],
        [0.0673],
        [0.0768],
        [0.0572],
        [0.0591],
        [0.0519],
        [0.0743],
        [0.0799],
        [0.0559],
        [0.0718],
        [0.1091],
        [0.0783],
        [0.0647],
        [0

In [19]:
X1_test = torch.Tensor(X1_test.values)
pred_y_test = RNET.predict(X1_test)
print(pred_y_test)

tensor([[0.0491],
        [0.0499],
        [0.0505],
        [0.0641],
        [0.0626],
        [0.0513],
        [0.0501],
        [0.0493],
        [0.0652],
        [0.0511],
        [0.0494],
        [0.0497],
        [0.0492],
        [0.0504],
        [0.0497],
        [0.0504],
        [0.0499],
        [0.0556],
        [0.0502],
        [0.0550],
        [0.0596],
        [0.0499],
        [0.0501],
        [0.0495],
        [0.0511],
        [0.0528],
        [0.0500],
        [0.0496],
        [0.0495],
        [0.0504],
        [0.0541],
        [0.0523],
        [0.0505],
        [0.0660],
        [0.0108],
        [0.0562],
        [0.0492],
        [0.0505],
        [0.0563],
        [0.0503],
        [0.0495],
        [0.0601],
        [0.0498],
        [0.0498],
        [0.0493],
        [0.0500],
        [0.0555],
        [0.0499],
        [0.0504],
        [0.0551],
        [0.0525],
        [0.0535],
        [0.0525],
        [0.0494],
        [0.0609],
        [0